In [1]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
import matplotlib.pyplot as plt
import os

fase = '01_LecturaDatos'

In [2]:
with open('gen_config.json', 'r') as file:
    gen_config =json.load(file)

In [3]:
folder = gen_config['folder']

path_group = gen_config['path_group']
path_prod_stats = gen_config['path_prod_stats']
path_prod_data = gen_config['path_prod_data']
#path_prod_data_group = gen_config['path_prod_data_group']
#path_prod_data_ungroup = gen_config['path_prod_data']
path_stock_data = gen_config['path_stock_data']

#periodo_rep_start = gen_config['var_periodo_rep_start']
#periodo_rep_end = gen_config['var_periodo_rep_end']

print(f"{'COMIENZA':-^100}")
print(f"{fase:-^100}")

----------------------------------------------COMIENZA----------------------------------------------
------------------------------------------01_LecturaDatos-------------------------------------------


In [4]:
df = pd.read_csv('../sell-in.txt',sep='\t')
prod_a_predecir = pd.read_csv('../productos_a_predecir.txt', sep='\t')
prod_data = pd.read_csv('../tb_productos_descripcion.txt', sep='\t')
stock_data = pd.read_csv('../tb_stocks.txt', sep='\t')

In [5]:
#filtrar los de la lista final
df = df[df['product_id'].isin(prod_a_predecir['product_id'])]

In [32]:
# prod_data_group = prod_data_ungroup.groupby(['cat1','cat2','cat3','brand','descripcion'],
#                             observed=False,
#                             as_index=False,
#                             group_keys=True
#                             ).count()
# prod_data_group['group_index'] = prod_data_group.index
# prod_data_group.rename(columns={'sku_size': 'sku_size_count', 'product_id': 'product_id_count'},
#                         inplace=True)

# prod_data_ungroup = prod_data_ungroup.merge(prod_data_group.drop(
#     columns=['sku_size_count','product_id_count']),
#     on=['cat1','cat2','cat3','brand','descripcion'],
#     how='left')

In [33]:
# for col in prod_data_group.select_dtypes(include=['object']).columns:
#     prod_data_group[col] = prod_data_group[col].astype('category')

# for col in prod_data_ungroup.select_dtypes(include=['object']).columns:
#     prod_data_ungroup[col] = prod_data_ungroup[col].astype('category')

In [34]:
df_grouped = df.groupby(['periodo', 'product_id', 'customer_id'])[['tn']].sum().reset_index()

In [35]:
#df_grouped = df_grouped.merge(prod_data_ungroup, on='product_id', how='left')

In [36]:
# print(f"Unicos:\n{df.nunique()}\n\n")
# print(f"Nulos:\n{df.isnull().sum()}")

In [37]:
# df_grouped = df_grouped.groupby(['periodo', 'group_index']).sum().reset_index()

In [38]:
# df_grouped['group_index'] = df_grouped['group_index'].astype(int)

In [39]:
# df_grouped = df_grouped[['group_index', 'periodo', 'tn']]
# df_grouped.sort_values(by=['group_index', 'periodo'], inplace=True)
# df_grouped.reset_index(drop=True, inplace=True)

In [40]:
# print(f"{'Tamaño data_grouped':.<25}: {df_grouped.shape}")
# df_grouped.head()

In [42]:
df_grouped

,periodo,product_id,customer_id,tn
0,201701,20001,10001,99.43861
1,201701,20001,10002,35.72806
2,201701,20001,10003,143.49426
3,201701,20001,10004,184.72927
4,201701,20001,10005,19.08407
...,...,...,...,...
2293476,201912,21267,10219,0.00131
2293477,201912,21276,10029,0.00075
2293478,201912,21276,10052,0.00594
2293479,201912,21276,10219,0.00075


In [43]:
# probar tambien agregar nievl producto para features

prod_stats = df_grouped.groupby(['product_id','customer_id']).apply(lambda group: pd.Series({
    'primer_periodo': group['periodo'].min(),
    'ultimo_periodo': group['periodo'].max(),
    'values': group['tn'].count(),
    'total_tn': group['tn'].sum(),
    'min_tn': group['tn'].min(),
    'average_tn': group['tn'].mean(),
    'median_tn': group['tn'].median(),
    'std_dev_tn': group['tn'].std(),
    'iqr_tn': (group['tn'].quantile(0.75) - group['tn'].quantile(0.25)),
    'max_tn': group['tn'].max()
}))
prod_stats['primer_periodo'] = prod_stats['primer_periodo'].astype(int)
prod_stats['primer_periodo'] = pd.to_datetime(prod_stats['primer_periodo'].astype(str), format='%Y%m')

prod_stats['ultimo_periodo'] = prod_stats['ultimo_periodo'].astype(int)
prod_stats['ultimo_periodo'] = pd.to_datetime(prod_stats['ultimo_periodo'].astype(str), format='%Y%m')

prod_stats.reset_index(inplace=True)

In [ ]:
# df_prod_level = df[['product_id', 'periodo', 'tn']].groupby(['product_id', 'periodo'], as_index=False).sum()
# df_prod_level.sort_values(by=['product_id', 'periodo'], inplace=True)

# df_prod_level = df_prod_level.merge(prod_data_ungroup[['product_id','group_index']], on='product_id', how='left')

# df_prod_level = df_prod_level[(df_prod_level['periodo'] >= periodo_rep_start) & (df_prod_level['periodo'] <= periodo_rep_end)]

# df_prod_level = df_prod_level.groupby(['product_id','group_index'],
#             as_index=False).apply(lambda group: pd.Series({
#                     'cuenta': group['periodo'].nunique(),
#                     'tn': group['tn'].sum(),}))

# df_prod_level['group_sum'] = df_prod_level.groupby('group_index')['tn'].transform('sum')
# df_prod_level['proporcion'] = df_prod_level['tn'] / df_prod_level['group_sum']

# df_prod_level.sort_values(by=['group_index', 'product_id'], inplace=True)
# #df_prod_level = df_prod_level.merge(prod_stats, on='group_index', how='left')

# df_prod_level = df_prod_level[['product_id','group_index','proporcion']]

# prod_data_ungroup = prod_data_ungroup.merge(df_prod_level, on=['product_id','group_index'], how='left')

In [44]:
print(f"{'Tamaño prod_stats':.<25}: {prod_stats.shape}")
# prod_stats.head()

Tamaño prod_stats........: (262805, 12)


In [45]:
if not os.path.exists(folder):
    os.makedirs(folder)

df_grouped.to_parquet(f'{folder}/{path_group}', index=False)
prod_stats.to_parquet(f'{folder}/{path_prod_stats}', index=False)
prod_data.to_parquet(f'{folder}/{path_prod_data}', index=False)
#prod_data_group.to_parquet(f'{folder}/{path_prod_data_group}', index=False)
#prod_data_ungroup.to_parquet(f'{folder}/{path_prod_data_ungroup}', index=False)
stock_data.to_parquet(f'{folder}/{path_stock_data}', index=False)

In [46]:
print(f"{fase:-^100}")
print(f"{'FINALIZA':-^100}\n\n\n")

------------------------------------------01_LecturaDatos-------------------------------------------
----------------------------------------------FINALIZA----------------------------------------------





In [77]:
# for col in prod_data_group.select_dtypes(include=['object']).columns:
#     prod_data_group[col] = prod_data_group[col].astype('category')

# for col in prod_data_ungroup.select_dtypes(include=['object']).columns:
#     prod_data_ungroup[col] = prod_data_ungroup[col].astype('category')